# Load Data

First we load data from HDFS. It is stored as a trivial CSV file with three columns
1. product name
2. review text
3. rating (1 - 5)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema =  StructType([
    StructField('name',StringType(),True),
    StructField('review',StringType(), True),
    StructField('rating',StringType(), True),
])

raw_data = spark.read.schema(schema).csv("s3a://dimajix-training/data/amazon_baby")

## Clean and Cache Data

We need to convert the "rating" columns to an integer - but this will obviously fail for the first record, as this one contains the CSV header. So we need to perform some cleanup after trying to convert the data.

For helping distributing the workload, we repartition the DataFrame and also cache it.

In [ ]:
data = raw_data.withColumn('rating',col('rating').cast(IntegerType())) \
    .filter(col('rating').isNotNull()) \
    .filter(col('review').isNotNull()) \
    .repartition(31) \
    .cache()

data.limit(5).toPandas()

# Split Train Data / Test Data

Now let's do the usual split of our data into a training data set and a validation data set. Let's use 80% of all reviews for training and 20% for validation

In [ ]:
train_data, test_data = ... # YOUR CODE HERE

print("train_data: %d" % train_data.count())
print("test_data: %d" % test_data.count())

# Implement Transformer

We need a custom Transformer to build the pipeline. The transformer should remove all punctuations from a given column containing text.

In [ ]:
from pyspark.ml import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

def remove_punctuations(text):
    import string
    for c in string.punctuation:
        text = text.replace(c, ' ')
    return text


class PunctuationCleanupTransformer(Transformer):
    def __init__(self, inputCol, outputCol):
        """
        Construction of PunctuationCleanupTransformer which takes two arguments:
        inputCol - name of input column
        outputCol - name of output column
        """
        super(Transformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol

    def _transform(self, dataset):
        """
        Protecetd _transform method which will be called by the public transform
        method. You should not call this method directly.
        """
        remove_punctuation_udf = udf(remove_punctuations, StringType())
        return dataset.withColumn(self.outputCol, remove_punctuation_udf(self.inputCol))

## Test Transformer

Lets create an instance of the Transformer and test it

In [ ]:
# Create instance of PunctuationCleanupTransformer and apply it to the data. The result should be stored in clean_data
cleaner = ...
clean_data = ...

# Extract a couple of rows, so we can inspect result
clean_data.limit(4).toPandas()

# Implement Transformer for Stemming

We need to stem words, and for doing so we use the Python NLTK library.

In [ ]:
from nltk.stem import PorterStemmer

def stem_word(words):
    ps = PorterStemmer()
    return [ps.stem(word) for word in words]


class PorterStemmerTransformer(Transformer):
    def __init__(self, inputCol, outputCol):
        """
        Constructor of PorterStemmerTransformer which takes two arguments:
        inputCol - name of input column
        outputCol - name of output column
        """
        super(Transformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol

    def _transform(self, dataset):
        """
        Protecetd _transform method which will be called by the public transform
        method. You should not call this method directly.
        """
        stem_word_udf = udf(stem_word, ArrayType(StringType()))
        return dataset.withColumn(self.outputCol, stem_word_udf(self.inputCol))

## Test Transformer

Again we want to test the `PorterStemmerTransformer`

In [ ]:
from pyspark.ml.feature import *

# First we need to Tokenize each line. In order to perform this task, we implement the following steps
# 1. Instantiate a Tokenizer instance from pyspark.ml.feature
# 2. Transform the raw data using the tokenizer
tokenizer = Tokenizer(inputCol='review', outputCol='words')
tokenized_data = ... # YOUR CODE HERE

# Then we can instantiate the PorterStemmerTransformer and use it on the words.
stemmer = ... # YOUR CODE HERE
stemmed_data = ... # YOUR CODE HERE

# Display first 4 entries of the result
stemmed_data.limit(4).toPandas()

# Create ML Pipeline

Now we have all components for creating an initial ML Pipeline. Remember that we have been using the following components before

* PunctuationCleanupTransformer - remove punctuations from reviews
* Tokenizer - for splitting reviews into words
* StopWordRemover - for removing stop words
* PorterStemmerTransformer - for stemming words
* NGram - for creating NGrams (we'll use two words per n-gram)
* CountVectorizer - for creating bag-of-word features from the words
* IDF - for creating TF-IDF features from the NGram counts
* LogisticRegression - for creating the real model

You also need to transform the incoming rating (1-5) to a sentiment (0 or 1) and you need to drop reviews with a rating of 3. This can be done using one ore more SQLTransformer instances. Inside the SQLTransformer instance you simply write SQL code and access the current DataFrame via `__THIS__`.

In [ ]:
from pyspark.ml.feature import *
from pyspark.ml.classification import *

# Define list of stopwords used in StopWordsRemover
stopWords = StopWordsRemover.loadDefaultStopWords("english")

stages = [
    # You will probably need in some meaningful order and with appropriate arguments
    #   CountVectorizer
    #   IDF
    #   LogisticRegression
    #   NGram
    #   PorterStemmerTransformer 
    #   PunctuationCleanupTransformer
    #   SQLTransformer
    #   StopWordsRemover
    #   Tokenizer
]

pipe = Pipeline(stages = stages)

# Fit Pipeline Model
Using training data, we create a PipelineModel by fitting the Pipeline to the training data

In [ ]:
# YOUR CODE HERE
model = ...

# Predict Data

Let us do some predictions of the test data using the model.

In [ ]:
# YOUR CODE HERE
pred = ...

pred.limit(10).toPandas()

# Model Evaluation
As in the original exercise, we want to use a custom metric for assessing the performance.

In [ ]:
from pyspark.ml.evaluation import *

class AccuracyClassificationEvaluator(Evaluator):
    def __init__(self, predictionCol='prediction', labelCol='label'):
        super(Evaluator,self).__init__()
        self.predictionCol = predictionCol
        self.labelCol = labelCol
    
    def _evaluate(self, dataset):
        num_total = dataset.count()
        num_correct = dataset.filter(col(self.labelCol) == col(self.predictionCol)).count()
        accuracy = float(num_correct) / num_total
        return accuracy

## Assess Performance

With the evaluator we can assess the performance of the prediction and easily compare it to a simple model which always predicts 'positive'.

In [ ]:
always_positive = pred.withColumn('prediction',lit(1.0))

evaluator = AccuracyClassificationEvaluator(predictionCol='prediction', labelCol='sentiment')

print("Model Accuracy = %f" % evaluator.evaluate(pred))
print("Baseline Accuracy = %f" % evaluator.evaluate(always_positive))

# Hyper Parameter Tuning

Again we want to tune some hyper parameters, but this time inside a pipeline. The methodology is the same as before, we can directly include the CrossValidator into the pipeline. But step by step...

First let us have a look at all paremeters of a LogisticRegression.

In [ ]:
print(LogisticRegression().explainParams())

## Create ParamGrid

Now we create a param grid that should be used for using different sets of parameters. We want to tweak two parameters again:

* the size of the n-grams should take values in [2,3,5]
* minDF of the CountVectorizer  should take values in [1, 2, 3, 5])

In order to create this grid, we first need to extract the corresponding stages from the previous pipeline.

In [ ]:
from pyspark.ml.tuning import *

# Extract specific pipeline stages. 
# YOUR CODE HERE
ngram = pipe.getStages()[...]
count = ...

# Create Parameter Grid
# YOUR CODE HERE
param_grid = ParamGridBuilder() \
    .addGrid(ngram.n, ...) \
    .addGrid(...) \
    .build()

## Create Pipeline

Now we can wrap the pipeline inside a CrossValidator. The CrossValidator works as a wrapper of the whole pipeline and selects the best parameters from the grid.

We want to put our own AccuracyClassificationValidator into the CrossValidator.

In [ ]:
evaluator = AccuracyClassificationEvaluator(labelCol='sentiment')
validator = CrossValidator(estimator=pipe, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Fit model to pipeline
model = ... # YOUR CODE HERE

In [ ]:
# Predict sentiment for test data
pred = ... # YOUR CODE HERE

# Evaluate and compare against baseline
print("Model Accuracy = %f" % evaluator.evaluate(pred))
print("Baseline Accuracy = %f" % evaluator.evaluate(always_positive))